[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/fast/fast_stable_diffusion.ipynb)

https://github.com/TheLastBen/fast-stable-diffusion

https://github.com/Van-wise/sd-colab


**如果遇到报错,请联系up：**

https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244

In [ ]:
#@title 👇 首次运行！
import os
import sys
import binascii
from IPython.utils import capture
from IPython.display import clear_output

webui_dir = '/content/sd'
gwebui_dir = '/content/drive/MyDrive/sd'

#Check GPU  ,不使用GPU会出现莫名其妙的错误哦。
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
!git config --global advice.detachedHead false
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("GPU is available")
else:
    sys.exit("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")

# git clone AUTOMATIC1111
!apt -y install -qq aria2 git
with capture.capture_output() as cap:
  !git clone --depth 1 -q --branch v1.4.0 https://github.com/Van-wise/sd
  %cd {webui_dir}
  !rm -rf webui.sh
  !mkdir -p {webui_dir}/cache
  os.environ['TRANSFORMERS_CACHE']=f"{webui_dir}/cache"
clear_output()

# Requirements stable-diffusion-stability-ai
with capture.capture_output() as cap:
  %cd /content
  !mkdir -p {webui_dir}/repositories/stable-diffusion-stability-ai
  !wget -q -i https://raw.githubusercontent.com/Van-wise/fast-sd/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !tar -C /content --zstd -xf sd_rep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt

  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  os.chdir('/content')

!pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv
clear_output()

#安装必备插件
%cd {webui_dir}/extensions
!git clone --depth 1 https://github.com/Van-wise/sd-localization-zh_Hans sd-localization-zh_Hans #中文
!git clone --depth 1 https://github.com/Van-wise/sd-infinite-image-browsing sd-infinite-image-browsing #图片浏览
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Van-wise/sd-colab/main/.env -d {webui_dir}/extensions/sd-infinite-image-browsing  -o .env
!git clone --depth 1 https://github.com/Van-wise/sd-controlnet sd-controlnet #controlnet
!git clone --depth 1 https://github.com/Van-wise/sd-prompt-all-in-one sd-prompt-all-in-one #提示词
!git clone --depth 1 https://github.com/Van-wise/sd-lycoris-a1111 sd-lycoris-a1111 #lycoris
!git clone --depth 1 https://github.com/etherealxx/batchlinks-webui batchlinks-webui #快捷下载

#安装自定义插件
#@markdown 自定义插件 (填写,插件文本下载地址)
extensions_link = "https://raw.githubusercontent.com/Van-wise/sd-colab/main/extensions.txt" #@param {type:"string"}
!wget {extensions_link} -O /content/extension.txt
with open('/content/extension.txt', 'r') as file:
    for line in file:
        extensions_url = line.strip()
        !git clone --depth 1 $extensions_url

!git clone https://huggingface.co/embed/negative /content/negative
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/negative/* {webui_dir}/embeddings && rm -rf /content/negative
!git clone https://huggingface.co/embed/lora /content/lora
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/lora/* {webui_dir}/models/Lora && rm -rf /content/lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {webui_dir}/models/ESRGAN -o 4x-UltraSharp.pth

#同步 秋葉aaaki Webui (可选)
%cd {webui_dir}
!wget -O "config.json" "https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/config.json"

clear_output()

#CKPT Model Download/Load
model_dir = f"{webui_dir}/models/Stable-diffusion"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -d {model_dir} -o chilloutmix_NiPrunedFp32Fix.safetensors

#vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d {webui_dir}/models/VAE -o vae-ft-mse-840000-ema-pruned.safetensors

#Lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/62833 -d {webui_dir}/models/Lora -o more_detail.safetensors

clear_output()

#链接 controlnet
#@markdown 选择ControlNet：
#@markdown 
#@markdown 0：不需要使用ControlNet
#@markdown 
#@markdown 1：下载ControlNet 模型到 Colab
controlnet = 1 #@param {type:"slider", min:0, max:1, step:1}
if controlnet == 1:
    # 下载ControlNet 模型到 Colab
    #ControlNet Model Download 
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_colab
    cndown_colab()
else:
    print("不使用 ControlNet、或已使用快捷方式。")

clear_output()

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    !wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/fast-sd/main/AUTOMATIC1111_files/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/Van-wise/sd/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/Van-wise/sd/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://github.com/Van-wise/fast-sd/raw/main/AUTOMATIC1111_files/blocks.py
  
    %cd {webui_dir}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/sd/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/sd/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/sd/modules/extras.py
    !sed -i 's@/content/gdrive/MyDrive/sd/stablediffusion@/content/sd/stablediffusion@' /content/sd/modules/paths.py
    !sed -i 's@print(\"No module.*@@' /content/sd/stablediffusion/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark"

#@markdown ####❎是否使用[ngrok](https://dashboard.ngrok.com/get-started/your-authtoken)网络加速
#@markdown ######填写ngrok_auth 的 token：
ngrok_auth=""  #@param {type:"string"}
if ngrok_auth:
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"

!python {webui_dir}/webui.py $wise


In [ ]:
#@title 👇启动 Stable-Diffusion
from IPython.utils import capture
import sys

with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    !wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  
    %cd {webui_dir}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {webui_dir}/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {webui_dir}/modules/extras.py
    !sed -i 's@{webui_dir}/repositories/stable-diffusion-stability-ai@/content/{sdw}/repositories/stable-diffusion-stability-ai/@' {webui_dir}/modules/paths.py
    !sed -i 's@print(\"No module.*@@' {webui_dir}/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py
    
wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark"   
!python {webui_dir}/webui.py $wise


In [ ]:
#下载模型到 SDwebui （colab）
#下载主模型，请根据自己的需求下载，修改下载地址和文件名即可
!wget https://civitai.com/api/download/models/11745 -O {webui_dir}/models/Stable-diffusion/Chilloutmix-Ni-pruned-fp32-fix.safetensors

#下载 Lora 模型
!wget https://civitai.com/api/download/models/16557 -O {webui_dir}/models/Lora/ShojoVibev_11.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {webui_dir}/models/Lora -o ShojoVibev_11.safetensors

In [ ]:
#下载模型到 GoogleDrive
#下载链接，文件名，models路径
url = "https://civitai.com/api/download/models/16557" #models文件下载链接
filename = "ShojoVibev_11.safetensors" #保存文件名 .safetensors or .pt
save_dir = f"{gwebui_dir}/models/Lora"  # f"{gwebui_dir}/models/Lora/Stable-diffusion/"

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o {filename} -d {save_dir} {url}